# はじめに

このノートではLightGBMを理解するために、下記のmeanxaiの動画を参考にしながら、LightGBMのアルゴリズムについて理解を深める。

- [[MXML-12-03] Light GBM [3/5] - Exclusive Feature Bundling (EFB), Greedy Bundling](https://www.youtube.com/watch?v=Y-IvfsjmqOQ&list=PLQXYdeUrQvu5z2hYI6TTiqBLCq7qWJ45n&index=4)
- [[MXML-12-04] Light GBM [4/5] - Merge Exclusive Features for EFB](https://www.youtube.com/watch?v=orSRRtWtPwE&list=PLQXYdeUrQvu5z2hYI6TTiqBLCq7qWJ45n&index=6)


今回は、Exclusive Feature Bundling(EFB)アルゴリズム、EFBに対するMerge Exclusive Featuresアルゴリズムに焦点をあてる。

## Exclusive Feature Bundling(EFB)アルゴリズム

EFBアルゴリズムは、特徴量の数を効果的にまとめてしまうことを提案しているアルゴリズム。具体的には、多くのスパースな特徴量は排他的で、互いに0以外の値を取っている(コンフリクト)ことは少ない。そのため、特徴量の多くがスパースなケースなどでは、個々の特徴から同じ特徴を持つようにまとめて(バンドル)しまえば、ロスレスで効率的に学習を進めることができる。

- Greedy Bundling: コンフリクトの回数が一定の閾値以下であればバンドルする
- Merge Exclusive Features: 同じバンドルに含まれる特徴量を、各値が取る範囲が被らないように範囲をシフトさせてまとめる

まずは、Greedy Bundlingから始める。下記のデータがあったとき、F0とF1はいくつかの値がコンフリクトしている(×で表現している)。6個がコンフリクトしている状態。

|No| F0| F1| F2| F3| F4|
|:--|:--|:--|:--|:--|:--|
|0|×1|×1|0|0|1|
|1|0|0|1|1|1|
|2|×1|×2|0|0|2|
|3|0|0|2|3|1|
|4|×2|×1|0|0|3|
|5|×3|×3|0|0|1|
|6|0|0|3|0|2|
|7|×1|×2|3|4|3|
|8|1|0|1|0|0|
|9|×2|×3|0|0|1|

F1とF2はほとんどが排他的な状態になっている(◯で表現している)。1個がコンフリクトしている状態。

|No| F0| F1| F2| F3| F4|
|:--|:--|:--|:--|:--|:--|
|0|1|◯1|◯0|0|1|
|1|0|◯0|◯1|1|1|
|2|1|◯2|◯0|0|2|
|3|0|◯0|◯2|3|1|
|4|2|◯1|◯0|0|3|
|5|3|◯3|◯0|0|1|
|6|0|◯0|◯3|0|2|
|7|1|2|3|4|3|
|8|1|◯0|◯1|0|0|
|9|2|◯3|◯0|0|1|

このように各特徴量ごとにコンフリクトの数を計算し、コンフリクト集計行列にまとめるとわかりやすい。

|| F0| F1| F2| F3| F4|
|:--|:--|:--|:--|:--|:--|
|F0| | 6 | 2 | 1 | 6 |
|F1|6 |  | 1 | 1 | 6 |
|F2|2 | 1 |  | 3 | 4 |
|F3|1 | 1 | 3 |  | 3 |
|F4| 6| 6 | 4 | 3 |  |
|$\sum$| 15| 14 | 10 | 8 | 19 |

そして、合計行を降順に並び替えたものをSearch Orderと呼ぶ。これを見るとF4は最も多くのコンフリクトが発生していることがわかる。

|| F4| F0| F1| F2| F3|
|:--|:--|:--|:--|:--|:--|
|$\sum$| 19| 15 | 14 | 10 | 8 |

これらの情報をもとにグラフにするとなおわかりよい。まずはコンフリクト許容値$K$を設定する。$K=1$とすることで、不要なパスを削除できる。F0とF3、F1とF2がバンドルされる。F3はすでにバンドルされているので、F1とバンドルされない。

<img src='./BoostBundle.png'>

|No| F0| F3|| F1| F2|| F4|
|:--|:--|:--|:--|:--|:--|:--|:--|
|0|1|0| |1|0| |1|
|1|0|1| |0|1| |1|
|2|1|0| |2|0| |2|
|3|0|3| |0|2| |1|
|4|2|0| |1|0| |3|
|5|3|0| |3|0| |1|
|6|0|0| |0|3| |2|
|7|1|4| |2|3| |3|
|8|1|0| |0|1| |0|
|9|2|0| |3|0| |1|

このような形でバンドルする特徴量を見つけていく。実際のアルゴリズムについては、文字での説明が難しいので、コードを用いて理解する。

## Exclusive Feature Bundling(EFB)アルゴリズムの実装

In [26]:
import numpy as np

x = np.array([[1, 1, 0, 0, 1],
              [0, 0, 1, 1, 1],
              [1, 2, 0, 0, 2],
              [0, 0, 2, 3, 1],
              [2, 1, 0, 0, 3],
              [3, 3, 0, 0, 1],
              [0, 0, 3, 0, 2],
              [1, 2, 3, 4, 3],
              [1, 0, 1, 0, 0],
              [2, 3, 0, 0, 2]])

# コンフリクト行列
n_row = x. shape [0]
n_col = x. shape [1]
conflictCnt = np.zeros((n_col, n_col))

# コンフリクトの数を数える。コンフリクトしていない場合、かけ合わせると0になる
for i in range(n_col):
  for j in range(i+1, n_col):
    conflictCnt[i, j] = len(np. where(x[:, i] * x[:, j] > 0)[0])

iu = np.triu_indices(n_col, 1)
il = (iu[1], iu[0])
conflictCnt[il] = conflictCnt[iu]

# search order行列を作る
degree = conflictCnt.sum(axis=0)
searchOrder = np.argsort(degree)[::-1] # 降順

# コンフリクトの許容値
K=1 
bundles, bundlesConflict = [], [] 

for i in searchOrder:
  needNew = True
  for j in range(len(bundles)): 
    # 最初はからなのでスキップされる
    cnt = conflictCnt[bundles[j][-1], i]

    # 許容値K以下をバンドルの候補とする
    if cnt + bundlesConflict[j] <= K:
      # i 番目の特徴量をj番目のバンドルに追加
      bundles[j].append(i)
      # j番目のバンドルに含まれる特徴量のコンフリクト数を更新
      bundlesConflict[j] += cnt
      needNew = False
      break

  if needNew:
    bundles.append([i])
    bundlesConflict.append(0.)

print('bundles:', bundles)
print ('bundlesConflict:', bundlesConflict)

bundles: [[4], [0, 3], [1, 2]]
bundlesConflict: [0.0, 1.0, 1.0]


## Merge Exclusive Featuresアルゴリズム

Merge Exclusive Featuresアルゴリズムについて。このアルゴリズムはバンドルされた個々の特徴量から同じ特徴を持つように構築するアルゴリズム。つまり、何でもかんでも合成すればよいのではなく、元の特徴を識別できるように構築する必要がある。ヒストグラムベースの分割アルゴリズムは連続値ではなく離散ビンで値を保存するため、排他的な特徴を異なるビンに移動して共存させることができる。

具体的には、オフセットを追加することで実現する。特徴量Aは範囲0-10で、特徴量Bは範囲0-20だとする。特徴量Bにオフセットとして10を追加する。オフセットは特徴量Aの最大値であり、バンドルされた特徴量は0−30の範囲をとるようになる。Greedy Bundlingの説明で使用したデータを利用する。`F = [1: F0, 2:F3]`である。まずはF0,F3の最大値を取得して、`binRange = [0, 3, 7(3+4)]`を作成する。2つ目の特徴量の最大値は1つ目の特徴量の最大値にプラスする。あとは、F0,F3の各値を比較して、`newBin`を作成する。2つ目の特徴量が0️のときは、1つ目の特徴量をそのまま持ってくる。反対に、1つ目の特徴量が0️のときは、2つ目の特徴量に1つ目の特徴量の最大範囲を足し合わせる。コンフリクトしている部分は、1つ目の特徴量の値に`binRange`の1つ目の値である0を足して、2つ目の特徴量の値に`binRange`の2つ目の値である3を足しあわせる。

|No| F0(j=1)| F3(j=2)||newBin|
|:--|:--|:--|:--|:--|
|i=1|1|0 | F[1].bin[1]=1≠0(F[2].bin[1]=0) → newBin[0]=1+0 |1|
|i=2|0|1 | 2つ目の特徴量の1は1つ目の特徴量の最大範囲を足すため1+3=4|4|
|i=3|1|0 | |1|
|i=4|0|3 | (F[1].bin[4]=0)F[2].bin[1]=3≠0 → newBin[4]=3+3 |6|
|i=5|2|0 | |2|
|i=6|3|0 | F[1].bin[6]=3≠0(F[2].bin[6]=0) → newBin[6]=3+0 |3|
|i=7|0|0 | |0|
|i=8|1|4 | F[1].bin[8]=1≠0(F[2].bin[8]=4≠0) → newBin[8]=1+0(newBin[8]=4+3) |7|
|i=9|1|0 | |1|
|i=10|2|0| F[1].bin[10]=2≠0(F[2].bin[10]=0) → newBin[10]=2+0 |2|

## Merge Exclusive Featuresアルゴリズムの実装


In [32]:
x = np.array([[1, 1, 0, 0, 1],
              [0, 0, 1, 1, 1],
              [1, 2, 0, 0, 2],
              [0, 0, 2, 3, 1],
              [2, 1, 0, 0, 3],
              [3, 3, 0, 0, 1],
              [0, 0, 3, 0, 2],
              [1, 2, 3, 4, 3],
              [1, 0, 1, 0, 0],
              [2, 3, 0, 0, 2]])


# Greedy Bundlingの結果
bundles = [[4], [0, 3], [1, 2]]

# Algorithm: Merge Exclusive Features 
def merge_features(numData, F): 
  binRanges = [0]
  totalBin = 0
  for f in F:
    totalBin += np.max(f) 
    binRanges.append(totalBin) 

  newBin = np.zeros(numData, dtype=int) 
  for i in range(numData):
    newBin[i] = 0
    for j in range(len(F)): 
      if F[j][i] != 0:
          newBin[i] = F[j][i] + binRanges[j]
  return newBin, binRanges
  
F = [x[:, i] for i in bundles[1]]
newBin, binRanges = merge_features(x.shape[0], F)
print('newBin:', newBin)
print('binRanges:', binRanges)

newBin: [1 4 1 6 2 3 0 7 1 2]
binRanges: [0, 3, 7]
